# Multi-label image classification with ImageDataGenerator

In [107]:
# sklearn
from sklearn.model_selection import train_test_split

# tensorflow and keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Input, Conv2D, Flatten, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

import datetime
import pandas as pd

In [ ]:

MAIN_PATH = "/home/jupyter/" # Path to the directory which contains CSVs and the folder 'dataset'
IMAGES = "dataset"
CSV_NAME = "wikiart-movement-genre_True-class_3-merge_test1-n_1000_max.csv" # Nwikiart-movement-genre_True-class_3-merge_test1-n_1000_max.csvme of the CSV we want to use
NUM_MOVEMENT = 3 # Number of movements to classify
NUM_GENRE = 10 # Number of genres to classify
IMG_HEIGHT = IMG_WIDTH = 224 # Model's inputs shapes

USER = "pablo" # Choose between 'common', 'pablo', 'quentin', 'gregoire', 'alex'
MODEL_NAME = "Custom v1" # Set the name of the model 

'''----------------------------------
Load the CSV
----------------------------------'''
BATCH_SIZE = 32
EPOCHS = 50


'''----------------------------------
Load the CSV
----------------------------------'''
raw_df = pd.read_csv(DIRECTORY + CSV_NAME)
df = raw_df.iloc[:, 0:3]
assert type(df) == type(pd.DataFrame()) # Check if we created a dataframe
assert df.iloc[:, 1].nunique() ==  NUM_MOVEMENT # Check if we have the correct number of movements
assert df.iloc[:, 2].nunique() ==  NUM_GENRE # Check if we have the correct number of genres

print(f"Number of images : {df.shape[0]}")

'''----------------------------------
Train, test, val split
----------------------------------'''
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)
df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True)
assert type(df_train) == type(pd.DataFrame()) # Check if we created dataframes
assert type(df_test) == type(pd.DataFrame())
assert type(df_val) == type(pd.DataFrame())

print(f"Number of images in train set : {df_train.shape[0]}")
print(f"Number of images in val set : {df_val.shape[0]}")
print(f"Number of images in test set : {df_test.shape[0]}")
print('\n')


'''----------------------------------
Setup outputs columns
----------------------------------'''
assert len(list(df.columns[1:])) == 2 # Check if we have two outputs columns
columns=list(df.columns[1:])


'''----------------------------------
Train ImageDataGenerator
----------------------------------'''
train_datagen = ImageDataGenerator( # This generator is only used to train data because it has data augmentation and we do not want to augment data from the test or val set
    rescale=1./255,
    rotation_range=15,
    zoom_range=(0.95, 0.95),
    horizontal_flip=True,
    dtype=tf.float32
    )

assert type(train_datagen) == type(ImageDataGenerator())

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train, # Dataset used to get the path (column filename) and the linked outputs
    directory=DIRECTORY + IMAGES, # Path to the images
    x_col="file_name", # Column with the name of the images that the generator will get from the directory
    y_col=columns, # Columns with the output of the images that the generator will get from the csv
    batch_size=BATCH_SIZE,
    seed=None,
    shuffle=True,
    class_mode="raw", # numpy array of values in y_col columns
    target_size=(IMG_HEIGHT, IMG_WIDTH), # Resize the images to the input shape of the model
    data_format='channels_last'
    ) 


'''----------------------------------
Test and Val ImageDataGenerator
----------------------------------'''
test_val_datagen = ImageDataGenerator(  # We use a new generator without data augmentation
    rescale=1./255
    )

val_generator = test_val_datagen.flow_from_dataframe(
    dataframe=df_val, 
    directory=DIRECTORY + IMAGES,
    x_col="file_name",
    y_col=columns,
    batch_size=BATCH_SIZE,
    seed=None,
    shuffle=False,
    class_mode="raw", 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    data_format='channels_last'
    )

test_generator = test_val_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=DIRECTORY + IMAGES,
    x_col="file_name",
    batch_size=1,
    seed=None,
    shuffle=False,
    class_mode=None, # No targets are returned (the generator will only yield batches of image data, which is useful to use in model.predict()
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    data_format='channels_last'
    )


assert type(test_val_datagen) == type(ImageDataGenerator())


In [ ]:
'''----------------------------------
Create model (Functional API)
----------------------------------'''
tf.keras.backend.clear_session()

# Model tout pourri pour l'instant, transfert learning ensuite
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32)
x = Conv2D(32, (3, 3), padding = 'same')(inputs)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64, (3, 3), padding = 'same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

movement_output = Dense(NUM_MOVEMENT, activation = 'softmax')(x) # First output for movement
genre_output = Dense(NUM_GENRE, activation = 'softmax')(x) # Second outptu for genre

model = Model(inputs, [movement_output,genre_output]) # Create the model with 2 outputs


In [ ]:
'''----------------------------------
Compile model
----------------------------------'''
model.compile(
    optimizer='adamax', 
    loss={'dense_1': keras.losses.CategoricalCrossentropy(), 
          'dense_2': keras.losses.CategoricalCrossentropy()},
    metrics={
        "dense_1": [
            keras.metrics.Accuracy(),
            keras.metrics.CategoricalAccuracy(),
            ],
        "dense_2": [
            keras.metrics.Accuracy(),
            keras.metrics.CategoricalAccuracy(),
            ]
        }
    )
# We have 2 losses one for each output

In [ ]:
'''----------------------------------
Setup callbacks and tensorboard
----------------------------------'''

es = EarlyStopping(monitor='val_loss', patience=11, mode='min', restore_best_weights=True)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=5, min_lr=1e-6)

%load_ext tensorboard
log_dir = f"logs/{USER}/{MODEL_NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tsboard = TensorBoard(log_dir=log_dir)

In [ ]:
'''----------------------------------
Fit model
----------------------------------'''
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size # see steps_per_epoch comment below
STEP_SIZE_VAL = val_generator.n//val_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size


def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(2)])

# The model does not see every image at each epoch !

history = model.fit(
    generator_wrapper(train_generator),
    steps_per_epoch=STEP_SIZE_TRAIN, # No 'batch_size' parameter for data coming from a generator or tf.Dataset, common choice is to use num_samples // batch_size
    epochs=EPOCHS,
    validation_data=generator_wrapper(val_generator), # We use a generator as the validation_data
    validation_steps=STEP_SIZE_VAL,
    callbacks=[es, rlrp, tsboard],
    verbose=1
    )


In [ ]:
'''----------------------------------
Evaluate
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.evaluate(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1
    )


In [ ]:
'''----------------------------------
Predict
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.predict(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1
    )
